5.1 如何使用LCEL

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

In [ ]:
from langchain.llms import Ollama

chat = Ollama(model="openchat:latest")

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "請回答以下提問: \n{query}"
)

In [ ]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat,
    prompt=prompt_template,
    output_parser=output_parser
)

output = chain.invoke({"query": "生成式人工智能是什麼?"})
print(output["text"])

In [ ]:
chain = prompt_template | chat | output_parser   #LCEL

output = chain.invoke({"query": "什麼是AI?"})

print(output)

5.2 創建AI智能體

In [ ]:
chain = prompt_template | chat | output_parser   #LCEL

output = chain.invoke({"query": "現在幾點?"})

print(output)

In [ ]:
#使用自定義tool
import datetime
from langchain.agents import tool

@tool
def check_time(format:str = "%Y-%m-%d %H:%M:%S"):
    """根據指定的格式返回當前日期和時間"""

    current_time = datetime.datetime.now()

    formatted_time = current_time.strftime(format)

    return formatted_time

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
# https://smith.langchain.com/hub/conquereraman/react-chat
from langchain import hub
prompt_template = hub.pull("aixgeek/react4chinese")

tools = [check_time]

agent = create_react_agent(
    llm=chat,
    tools=tools,
    prompt=prompt_template
)

agent_executer = AgentExecutor(
    llm=chat,
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

agent_executer.invoke({"input": "現在幾點?"})

5.3 AI Agent如何使用LangChain自帶工具

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

In [ ]:
#使用地端model
from langchain.llms import Ollama

chat = Ollama(model="openchat:latest", temperature=0.1)

In [ ]:
#使用langchain內建tool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

prompt = hub.pull("aixgeek/react4chinese")

In [ ]:
api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,            #最多回傳多少搜尋結果
    doc_content_chars_max=500   #最大字符串
)

wikitool = WikipediaQueryRun(
    api_wrapper=api_wrapper
)

tools = [wikitool]

In [ ]:
agent = create_react_agent(
    llm=chat,
    tools=tools,
    prompt=prompt
)

agent_executer = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
input = "現任香港特首是誰?"

agent_executer.invoke({"input": input})

改使用雲端model: OpenAI

In [ ]:
#改使用雲端model: OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.1
)

In [ ]:
#使用langchain內建tool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
#from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents import create_openai_functions_agent, AgentExecutor   #改成使用OpenAI創建agent的方式
from langchain import hub

#prompt = hub.pull("aixgeek/react4chinese")
# set the LANGCHAIN_API_KEY environment variable (create key in settings) 
# https://smith.langchain.com/hub/brianxiadong/openai-functions-agent
prompt = hub.pull("brianxiadong/openai-functions-agent")                    #改成使用OpenAI創建agent的prompt template

In [ ]:
api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,            #最多回傳多少搜尋結果
    doc_content_chars_max=500   #最大字符串
)

wikitool = WikipediaQueryRun(
    api_wrapper=api_wrapper
)

tools = [wikitool]

In [ ]:
agent = create_openai_functions_agent(
    llm=chat,
    tools=tools,
    prompt=prompt
)

agent_executer = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
input = "現任香港特首是誰?"
system = "請用中文思考與回答問題"
chat_history = []

agent_executer.invoke({"input": input, "system": system, "chat_history": chat_history})